In [2]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../datasets/BankChurners.csv')
data = data.iloc[:,1:21]
np.random.seed(42)

In [4]:
object_columns = data.select_dtypes('object').columns
for i in object_columns:
    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']

category : [0 1]
classes : ['F' 'M']

category : [0 1 2 3 4 5 6]
classes : ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

category : [0 1 2 3]
classes : ['Divorced' 'Married' 'Single' 'Unknown']

category : [0 1 2 3 4 5]
classes : ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']

category : [0 1 2 3]
classes : ['Blue' 'Gold' 'Platinum' 'Silver']



## Under Sampling

In [5]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

random.seed(42)

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()
    
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Logistic Model's score by Under sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.942833,0.965862,0.920875,0.901777
OneSidedSelection,0.942529,0.965274,0.920831,0.901283
NeighbourhoodCleaningRule,0.940799,0.949382,0.932370,0.899803
EditedNearestNeighbours,0.938344,0.940553,0.936145,0.896347
AllKNN,0.935379,0.932902,0.937870,0.891905
RepeatedEditedNearestNeighbours,0.934441,0.927016,0.941986,0.890918
NearMiss,0.858161,0.779871,0.953924,0.783810


## Over Sampling

In [6]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

random.seed(42)

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        lr = LogisticRegression(random_state=42)
        lr.fit(x_train_ss, y_train)
        pred = lr.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Logistic Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by Over sampling


,f1_score,recall,precision,accuracy
SMOTE(0.3),0.940868,0.950559,0.931373,0.899803
ADASYN(0.3),0.940251,0.949382,0.931293,0.898815
BorderlineSMOTE(0.3),0.939660,0.944085,0.935277,0.898322
SMOTE(0.4),0.935111,0.928782,0.941527,0.891905
BorderlineSMOTE(0.4),0.932776,0.918776,0.947209,0.888944
ADASYN(0.4),0.932541,0.923484,0.941777,0.887957
SMOTE(0.5),0.932056,0.916421,0.948234,0.887957
SMOTE(0.6),0.930317,0.907593,0.954208,0.885982
BorderlineSMOTE(0.5),0.927816,0.904061,0.952854,0.882034
ADASYN(0.5),0.926859,0.898764,0.956767,0.881046


## Combine Sampling

In [7]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Logistic Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek,0.909767,0.863449,0.961337,0.856367
SMOTEENN,0.880463,0.806357,0.969568,0.816387


## Pipeline을 통한 Combine Sampling

In [8]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

lr = LogisticRegression()

random.seed(42)

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, lr)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTEOneSidedSelection(0.3),0.941074,0.949382,0.932909,0.900296
SMOTETomekLinks(0.3),0.940765,0.948793,0.932870,0.899803
ADASYNOneSidedSelection(0.3),0.939217,0.945851,0.932676,0.897335
ADASYNTomekLinks(0.3),0.939217,0.945851,0.932676,0.897335
BorderlineSMOTEOneSidedSelection(0.3),0.937720,0.939376,0.936070,0.895360
BorderlineSMOTETomekLinks(0.3),0.937720,0.939376,0.936070,0.895360
SMOTEOneSidedSelection(0.4),0.935111,0.928782,0.941527,0.891905
SMOTETomekLinks(0.4),0.935111,0.928782,0.941527,0.891905
SMOTENeighbourhoodCleaningRule(0.3),0.934089,0.921719,0.946796,0.890918
ADASYNNeighbourhoodCleaningRule(0.3),0.933014,0.918187,0.948328,0.889437


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [9]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.942833,0.965862,0.920875,0.901777
OneSidedSelection,0.942529,0.965274,0.920831,0.901283
SMOTEOneSidedSelection(0.3),0.941074,0.949382,0.932909,0.900296
NeighbourhoodCleaningRule,0.940799,0.949382,0.932370,0.899803
SMOTETomekLinks(0.3),0.940765,0.948793,0.932870,0.899803
ADASYNOneSidedSelection(0.3),0.939217,0.945851,0.932676,0.897335
ADASYNTomekLinks(0.3),0.939217,0.945851,0.932676,0.897335
EditedNearestNeighbours,0.938344,0.940553,0.936145,0.896347
BorderlineSMOTEOneSidedSelection(0.3),0.937720,0.939376,0.936070,0.895360
BorderlineSMOTETomekLinks(0.3),0.937720,0.939376,0.936070,0.895360


## NO Sampling

In [10]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
NO_Sampling = pd.DataFrame(columns=columns)

random.seed(42)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)


lr = LogisticRegression(random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

NO_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
NO_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
NO_Sampling.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.929156,0.968805,0.892625,0.876111
